In [41]:
import os.path

import pandas as pd
columns = ['cell_id', 'num_packets', 'pcap_total_size', 'server_sent_pkg_size_sum', 'server_avg_pkg_size',
           'server_num_pkg_sent', 'client_sent_pkg_size_sum', 'client_avg_pkg_size',
           'client_num_pkg_sent']
df = pd.DataFrame(columns=columns)

In [45]:
from os import listdir
from os.path import isfile, join, getsize
import scapy.all as S

for i in range(1,100):
    path=f"./filtered/cell_{i}/"
    files = [join(path,f) for f in listdir(path) if isfile(join(path, f))]
    for file in files:
        local_dic = {'cell_id': i}
        f = open(file,'rb')
        local_dic['pcap_total_size'] = getsize(file)
        f.close()
        packets = S.rdpcap(file)
        local_dic['num_packets'] =  len(packets)
        server_pkt_count = 0
        client_pkt_count = 0
        server_pkt_size_sum = 0
        client_pkt_size_sum = 0
        for packet in packets:
            ip_layer = packet.getlayer('IP').fields
            pkt_size = int(packet.sprintf('%IP.len%'))
            if '172' in ip_layer['src']:
                client_pkt_size_sum += pkt_size
                client_pkt_count += 1
            else:
                server_pkt_size_sum += pkt_size
                server_pkt_count += 1
        local_dic['server_sent_pkg_size_sum'] = server_pkt_size_sum
        local_dic['client_sent_pkg_size_sum'] = server_pkt_size_sum
        local_dic['client_num_pkg_sent'] = client_pkt_count
        local_dic['server_num_pkg_sent'] = server_pkt_count
        if (client_pkt_count != 0):
            local_dic['client_avg_pkg_size'] = client_pkt_size_sum/client_pkt_count
        if (server_pkt_count != 0):
            local_dic['server_avg_pkg_size'] = server_pkt_size_sum/server_pkt_count
        df = df.append(local_dic, ignore_index=True)
df.to_csv('features.csv', index=False)

